In [ ]:
import sys

sys.path[1:1] = ["_common", "_common/qiskit"]
sys.path[1:1] = ["../../_common", "../../_common/qiskit"]
sys.path[1:1] = ["../qiskit"]
import execute as ex
import metrics as metrics
from hamiltonian_simulation_kernel import HamiltonianSimulation, initial_state
from hamiltonian_simulation_exact import HamiltonianSimulationExact

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import TimeEvolutionProblem, SciPyRealEvolver

import time
import math
import numpy as np

import execute as ex
import metrics as metrics

from collections import defaultdict

np.random.seed(0)

In [ ]:
def construct_TFIM_hamiltonian(n_spins: int) -> SparsePauliOp:
    """
    Construct the Transverse Field Ising Model (TFIM) Hamiltonian.

    Args:
        n_spins (int): Number of spins (qubits).

    Returns:
        SparsePauliOp: The Hamiltonian represented as a sparse Pauli operator.
    """
    pauli_strings = []
    coefficients = []
    g = 1  # Strength of the transverse field

    # Pauli spin vector product terms
    for i in range(n_spins):
        x_term = 'I' * i + 'X' + 'I' * (n_spins - i - 1)
        pauli_strings.append(x_term)
        coefficients.append(g)

    identity_string = ['I'] * n_spins

    # ZZ operation on each pair of qubits in a linear chain
    for j in range(2):
        for i in range(j % 2, n_spins - 1, 2):
            zz_term = identity_string.copy()
            zz_term[i] = 'Z'
            zz_term[(i + 1) % n_spins] = 'Z'
            zz_term = ''.join(zz_term)
            pauli_strings.append(zz_term)
            coefficients.append(1.0)

    return SparsePauliOp.from_list(zip(pauli_strings, coefficients))

def generate_two_qubit_paulis(n_spins, pauli):
    identity_string = ['I'] * n_spins

    pauli_strings = []
    coefficients = []

    for j in range(2):
        for i in range(j % 2, n_spins - 1, 2):
            paulipauli_term = identity_string.copy()

            paulipauli_term[i] = pauli
            paulipauli_term[(i + 1) % n_spins] = pauli

            pauli_strings.append(''.join(paulipauli_term))
            coefficients.append(1.0)

    return pauli_strings, coefficients

def construct_heisenberg_hamiltonian(n_spins: int, w: int, hx: list[float], hz: list[float]) -> SparsePauliOp:
    """
    Construct the Heisenberg Hamiltonian with disorder.

    Args:
        n_spins (int): Number of spins (qubits).
        w (float): Strength of two-qubit interactions for heisenberg hamiltonian. 
        hx (list[float]): Strength of internal disorder parameter for heisenberg hamiltonian. 
        hz (list[float]): Strength of internal disorder parameter for heisenberg hamiltonian. 

    Returns:
        SparsePauliOp: The Hamiltonian represented as a sparse Pauli operator.
    """

    pauli_strings = []
    coefficients = []

    # Disorder terms
    for i in range(n_spins):
        x_term = 'I' * i + 'X' + 'I' * (n_spins - i - 1)
        z_term = 'I' * i + 'Z' + 'I' * (n_spins - i - 1)
        pauli_strings.append(x_term)
        coefficients.append(w * hx[i])
        pauli_strings.append(z_term)
        coefficients.append(w * hz[i])

    # Interaction terms

    for pauli in ['X','Y','Z']:
        pauli_string, coefficient = generate_two_qubit_paulis(n_spins, pauli)

        pauli_strings.extend(pauli_string)
        coefficients.extend(coefficient)

    return SparsePauliOp.from_list(zip(pauli_strings, coefficients))

In [ ]:
import json
from qiskit_aer import Aer
from qiskit import transpile

# Parameters for the Heisenberg Hamiltonian simulation
w = 1 
k = 5
t = .2 # Heisenberg state evolves "just" enough 

min_qubits = 2
max_qubits = 12

# Backend for simulation
backend = Aer.get_backend("qasm_simulator")

# Dictionary to store precalculated data
precalculated_data = {}

# store parameters in precalculated data
precalculated_data["w"] = w
precalculated_data["k"] = k
precalculated_data["t"] = t

# add parameter random values to precalculated data to ensure consistency
np.random.seed(26)
precalculated_data['hx'] = list(2 * np.random.random(20) - 1) # random numbers between [-1, 1]
np.random.seed(75)
precalculated_data['hz'] = list(2 * np.random.random(20) - 1) # random numbers between [-1, 1]

# Number of shots for the (close to) perfect circuit simulation
num_shots = 100000

# Generate precalculated data for each number of qubits
for n_spins in range(min_qubits, max_qubits+1):

        print(f"Now running n_spins {n_spins}")

        hx = precalculated_data['hx'][:n_spins]
        hz = precalculated_data['hz'][:n_spins]

        # Initialize the Hamiltonian simulation circuits
        qc = HamiltonianSimulation(n_spins, k, t, hamiltonian="heisenberg", w=w, hx = hx, hz = hz)
        qc2 = HamiltonianSimulation(n_spins, k, t, hamiltonian="tfim", w=w, hx = hx, hz = hz)

        # Transpile and run the circuits
        transpiled_qc = transpile(qc, backend, optimization_level=0)
        job = backend.run(transpiled_qc, shots=num_shots)
        result = job.result()
        counts = result.get_counts(qc)

        transpiled_qc2 = transpile(qc2, backend, optimization_level=0)
        job2 = backend.run(transpiled_qc2, shots=num_shots)
        result2 = job2.result()
        counts2 = result2.get_counts()

        # Normalize probabilities for Heisenberg model circuit 
        dist = {}
        for key in counts.keys():
            prob = counts[key] / num_shots
            dist[key] = prob

        # Normalize probabilities for TFIM model circuit
        dist2 = {}
        for key in counts2.keys():
            prob = counts2[key] / num_shots
            dist2[key] = prob

        # add dist values to precalculated data for use in fidelity calculation
        precalculated_data[f"Heisenberg - Qubits{n_spins}"] = dist  

    
        init_state = initial_state(n_spins, "checkerboard")
        hamiltonian_exact = construct_heisenberg_hamiltonian(n_spins, w, hx, hz)
        precalculated_data[f"Exact Heisenberg - Qubits{n_spins}"] = HamiltonianSimulationExact(hamiltonian_exact, t, init_state)
        
        precalculated_data[f"TFIM - Qubits{n_spins}"] = dist2

        init_state = initial_state(n_spins, "ghz")
        hamiltonian_exact = construct_TFIM_hamiltonian(n_spins)
        precalculated_data[f"Exact TFIM - Qubits{n_spins}"] = HamiltonianSimulationExact(hamiltonian_exact, t, init_state) 

# Save precalculated data to a JSON file
# https://stackoverflow.com/questions/36021332/how-to-prettyprint-human-readably-print-a-python-dict-in-json-format-double-q
with open('precalculated_data.json', 'w') as f:
    f.write(json.dumps(
        precalculated_data,
        sort_keys=True,
        indent=4,
        separators=(',', ': ')
        ))